In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.SILENT)

# Clear variables (equivalent to MATLAB's clear P T E A solver AvgTable)
P = None
T = None 
E = None
A = None
solver = None
AvgTable = None

In [2]:
print('This example illustrates the execution on a layered queueing network model.')
print('Performance indexes now refer to processors, tasks, entries, and activities.')
print('Indexes refer to the submodel (layer) where the processor or task acts as a server.')
print('NaN indexes indicate that the metric is not supported by the node type.')

# cwd = os.path.dirname(os.path.abspath(__file__))
# model = LayeredNetwork.parse_xml(os.path.join(cwd, 'example_layeredModel_1.xml'))

# Create model following MATLAB exactly
model = LayeredNetwork('myLayeredModel')

# MATLAB: P{1} = Processor(model, 'P1', 1, SchedStrategy.PS);
# MATLAB: P{2} = Processor(model, 'P2', 1, SchedStrategy.PS);
P = {}
P[1] = Processor(model, 'P1', 1, SchedStrategy.PS)
P[2] = Processor(model, 'P2', 1, SchedStrategy.PS)

# MATLAB: T{1} = Task(model, 'T1', 10, SchedStrategy.REF).on(P{1}).set_think_time(Exp.fit_mean(100));
# MATLAB: T{2} = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P{2}).set_think_time(Immediate());
T = {}
T[1] = Task(model, 'T1', 10, SchedStrategy.REF).on(P[1]).set_think_time(Exp.fit_mean(100))
T[2] = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P[2]).set_think_time(Immediate())

# MATLAB: E{1} = Entry(model, 'E1').on(T{1});
# MATLAB: E{2} = Entry(model, 'E2').on(T{2});
E = {}
E[1] = Entry(model, 'E1').on(T[1])
E[2] = Entry(model, 'E2').on(T[2])

# MATLAB: A{1} = Activity(model, 'AS1', Exp.fit_mean(1.6)).on(T{1}).boundTo(E{1});
# MATLAB: A{2} = Activity(model, 'AS2', Immediate()).on(T{1}).synchCall(E{2},1);
# MATLAB: A{3} = Activity(model, 'AS3', Exp.fit_mean(5)).on(T{2}).boundTo(E{2});
# MATLAB: A{4} = Activity(model, 'AS4', Exp.fit_mean(1)).on(T{2}).repliesTo(E{2});
A = {}
A[1] = Activity(model, 'AS1', Exp.fit_mean(1.6)).on(T[1]).bound_to(E[1])
A[2] = Activity(model, 'AS2', Immediate()).on(T[1]).synch_call(E[2], 1)
A[3] = Activity(model, 'AS3', Exp.fit_mean(5)).on(T[2]).bound_to(E[2])
A[4] = Activity(model, 'AS4', Exp.fit_mean(1)).on(T[2]).replies_to(E[2])

# MATLAB: T{1}.add_precedence(ActivityPrecedence.Serial(A{1}, A{2}));
# MATLAB: T{2}.add_precedence(ActivityPrecedence.Serial(A{3}, A{4}));
T[1].add_precedence(ActivityPrecedence.Serial(A[1], A[2]))
T[2].add_precedence(ActivityPrecedence.Serial(A[3], A[4]))

This example illustrates the execution on a layered queueing network model.
Performance indexes now refer to processors, tasks, entries, and activities.
Indexes refer to the submodel (layer) where the processor or task acts as a server.
NaN indexes indicate that the metric is not supported by the node type.


In [3]:
# MATLAB: options = LQNS.defaultOptions;
# MATLAB: options.keep = true; % uncomment to keep the intermediate XML files generates while translating the model to LQNS
options = LQNS.default_options()
options.keep = True  # uncomment to keep the intermediate XML files generates while translating the model to LQNS

# MATLAB: solver{1} = LQNS(model);
# MATLAB: AvgTable{1} = solver{1}.avg_table();
# MATLAB: AvgTable{1}
solver = {}
AvgTable = {}

if LQNS.isAvailable():
    solver[1] = LQNS(model)
    AvgTable[1] = solver[1].avg_table()
    print('AvgTable[1]:')
    print(AvgTable[1])
else:
    print('LQNS solver not available')

# MATLAB: CallAvgTable = {};
# MATLAB: [AvgTable{2}, CallAvgTable{2}] = solver{1}.get_raw_avg_tables();
# MATLAB: AvgTable{2}
# MATLAB: CallAvgTable{2}
CallAvgTable = {}
if LQNS.isAvailable():
    try:
        AvgTable[2], CallAvgTable[2] = solver[1].get_raw_avg_tables()
        print('AvgTable[2]:')
        print(AvgTable[2])
        print('CallAvgTable[2]:')
        print(CallAvgTable[2])
    except:
        print('getRawAvgTables method not yet implemented')

# MATLAB: AvgTable{3} = LN(model).get_avg_table
AvgTable[3] = LN(model).avg_table()
print('AvgTable[3]:')
print(AvgTable[3])

Node   QLen   Util   RespT   Tput
  P1 0.0000 0.1421  0.0000 0.0000
  T1 1.1215 0.1421  0.0000 0.0888
  E1 1.1215 0.1421 12.6310 0.0888
 AS1 0.1627 0.1421  1.8327 0.0888
 AS2 0.9587 0.0000 10.7983 0.0888
  P2 0.0000 0.5327  0.0000 0.0000
  T2 0.5327 0.5327  0.0000 0.0888
  E2 0.5327 0.5327  6.0000 0.0888
 AS3 0.4439 0.4439  5.0000 0.0888
 AS4 0.0888 0.0888  1.0000 0.0888
AvgTable[1]:
  Node    QLen    Util    RespT    Tput
0   P1  0.0000  0.1421   0.0000  0.0000
1   T1  1.1215  0.1421   0.0000  0.0888
2   E1  1.1215  0.1421  12.6310  0.0888
3  AS1  0.1627  0.1421   1.8327  0.0888
4  AS2  0.9587  0.0000  10.7983  0.0888
5   P2  0.0000  0.5327   0.0000  0.0000
6   T2  0.5327  0.5327   0.0000  0.0888
7   E2  0.5327  0.5327   6.0000  0.0888
8  AS3  0.4439  0.4439   5.0000  0.0888
9  AS4  0.0888  0.0888   1.0000  0.0888
getRawAvgTables method not yet implemented
Node  NodeType   QLen   Util   RespT  ResidT   ArvR   Tput
  P1 Processor 0.1645 0.1435  1.8331  1.8331 0.0897 0.0897
  P2 Process